# Projeto NLP

Existem muitas transformações de recursos que precisam ser feitas em dados de texto para levá-los a um ponto que os algoritmos de aprendizado de máquina possam entender. Felizmente, o Spark colocou os mais importantes em chamadas convenientes do Feature Transformer.

Vamos examiná-los antes de entrar no projeto.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

## Tokenizer
<p> <a href="http://en.wikipedia.org/wiki/Lexical_analysis#Tokenization"> Tokenização </a> é o processo de pegar um texto (como uma frase) e dividi-lo em termos individuais (geralmente palavras). Uma classe simples de <a href="api/scala/index.html#org.apache.spark.ml.feature.Tokenizer"> Tokenizer </a> fornece essa funcionalidade. O exemplo abaixo mostra como dividir frases em sequências de palavras. </p>

<p><a href="api/scala/index.html#org.apache.spark.ml.feature.RegexTokenizer">RegexTokenizer</a>permite mais
  tokenização avançada com base na correspondência de expressão regular (regex).
Por padrão, o parâmetro &#8220;pattern&#8221; (regex, default: <code>"\\s+"</code>)é usado como delimitadores para dividir o texto de entrada.
 Alternativamente, os usuários podem definir parâmetros &#8220;gaps&#8221; para falso indicando o regex &#8220;pattern&#8221; denota
 &#8220;tokens&#8221; em vez de dividir as lacunas e encontrar todas as ocorrências correspondentes como o resultado da tokenização.</p>

In [3]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [4]:
sentenceDataFrame = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

In [5]:
sentenceDataFrame.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [6]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

regexTokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W")
# alternativamente, pattern="\\w+", gaps(False)

countTokens = udf(lambda words: len(words), IntegerType())

tokenized = tokenizer.transform(sentenceDataFrame)
tokenized.select("sentence", "words")\
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

regexTokenized = regexTokenizer.transform(sentenceDataFrame)
regexTokenized.select("sentence", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

+-----------------------------------+------------------------------------------+------+
|sentence                           |words                                     |tokens|
+-----------------------------------+------------------------------------------+------+
|Hi I heard about Spark             |[hi, i, heard, about, spark]              |5     |
|I wish Java could use case classes |[i, wish, java, could, use, case, classes]|7     |
|Logistic,regression,models,are,neat|[logistic,regression,models,are,neat]     |1     |
+-----------------------------------+------------------------------------------+------+

+-----------------------------------+------------------------------------------+------+
|sentence                           |words                                     |tokens|
+-----------------------------------+------------------------------------------+------+
|Hi I heard about Spark             |[hi, i, heard, about, spark]              |5     |
|I wish Java could use case cla


## Remoção Stop Words 

<p><a href="https://en.wikipedia.org/wiki/Stop_words">Stop words</a>são palavras que
deve ser excluído da entrada, normalmente porque as palavras aparecem
freqüentemente e não carregam tanto significado.</p>

<p><code>StopWordsRemover</code>toma como entrada uma sequência de strings (por exemplo, a saída
de um <a href="ml-features.html#tokenizer">Tokenizer</a>)e deixa cair toda a parada
palavras das sequências de entrada. A lista de palavras irrelevantes é especificada por
um <code>stopWords</code> parâmetro. Palavras de parada padrão para alguns idiomas são acessíveis
chamando <code>StopWordsRemover.loadDefaultStopWords(language)</code>, para qual disponível
opções são &#8220;danish&#8221;, &#8220;dutch&#8221;, &#8220;english&#8221;, &#8220;finnish&#8221;, &#8220;french&#8221;, &#8220;german&#8221;, &#8220;hungarian&#8221;, 
&#8220;italian&#8221;, &#8220;norwegian&#8221;, &#8220;portuguese&#8221;, &#8220;russian&#8221;, &#8220;spanish&#8221;, &#8220;swedish&#8221; and &#8220;turkish&#8221;. 
Um parâmetro booleano <code> caseSensitive </code> indica se as correspondências devem ser sensíveis a maiúsculas e minúsculas
(falso por padrão).</p>

In [7]:
from pyspark.ml.feature import StopWordsRemover

sentenceData = spark.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "raw"])

remover = StopWordsRemover(inputCol="raw", outputCol="filtered")
remover.transform(sentenceData).show(truncate=False)


+---+----------------------------+--------------------+
|id |raw                         |filtered            |
+---+----------------------------+--------------------+
|0  |[I, saw, the, red, balloon] |[saw, red, balloon] |
|1  |[Mary, had, a, little, lamb]|[Mary, little, lamb]|
+---+----------------------------+--------------------+



## n-grams

Um n-grama é uma sequência de nn tokens (normalmente palavras) para algum inteiro nn. A classe NGram pode ser usada para transformar recursos de entrada em nn-gramas.

<p><code>NGram</code>leva como entrada uma sequência de strings (por exemplo, a saída de um <a href="ml-features.html#tokenizer">Tokenizer</a>).  O parâmetro <code> n </code> é usado para determinar o número de termos em cada $ n $ -grama. A saída consistirá em uma seqüência de $ n $ -gramas onde cada $ n $ -grama é representado por uma string delimitada por espaço de $ n $ palavras consecutivas. Se a sequência de entrada contiver menos de <code> n </code> strings, nenhuma saída será produzida.</p>


In [8]:
from pyspark.ml.feature import NGram

wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

ngramDataFrame = ngram.transform(wordDataFrame)
ngramDataFrame.select("ngrams").show(truncate=False)

+------------------------------------------------------------------+
|ngrams                                                            |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



_______
# Extrator de Feature
_______

<h2 id="tf-idf">TF-IDF</h2>

<p><a href="http://en.wikipedia.org/wiki/Tf%E2%80%93idf">Freqüência de documento de freqüência inversa de termo (TF-IDF) </a>
é um método de vetorização de recursos amplamente utilizado na mineração de texto para refletir a importância de um termo
a um documento no corpus. Denote um termo por <code> $ t $ </code>, um documento por d e o corpus por D.
A frequência do termo <code> $ TF (t, d) $ </code> é o número de vezes que o termo <code> $ t $ </code> aparece no documento <code> $ d $ </code>, enquanto
frequência do documento <code> $ DF (t, D) $ </code> é o número de documentos que contém o termo <code> $ t $ </code>. Se nós apenas usarmos
frequência de termos para medir a importância, é muito fácil enfatizar demais os termos que parecem muito
frequentemente, mas trazem poucas informações sobre o documento, por exemplo, &#8220;a&#8221;, &#8220;the&#8221;, e &#8220;of&#8221;. Se um termo aparecer
muitas vezes em todo o corpus, isso significa não traz informações especiais sobre um determinado documento.
A frequência inversa do documento é uma medida numérica de quanta informação um termo fornece:

$$ IDF(t, D) = \log \frac{|D| + 1}{DF(t, D) + 1} $$

onde | D | é o número total de documentos no corpus. Uma vez que o logaritmo é usado, se um termo
aparece em todos os documentos, seu valor IDF torna-se 0. Observe que um termo de suavização é aplicado para evitar
dividindo por zero para termos fora do corpus. A medida TF-IDF é simplesmente o produto de TF e IDF:

$$ TFIDF(t, d, D) = TF(t, d) \cdot IDF(t, D). $$


In [9]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [10]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [11]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternativamente, CountVectorizer também pode ser usado para obter vetores de frequência de termo

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[0,5,9,17],[0...|
|  0.0|(20,[2,7,9,13,15]...|
|  1.0|(20,[4,6,13,15,18...|
+-----+--------------------+



## CountVectorizer
CountVectorizer e CountVectorizerModel visam ajudar a converter uma coleção de documentos de texto em vetores de contagens de tokens. Quando um dicionário a priori não está disponível, CountVectorizer pode ser usado como um Estimator para extrair o vocabulário e gera um CountVectorizerModel. O modelo produz representações esparsas para os documentos sobre o vocabulário, que podem então ser passadas para outros algoritmos como LDA.

Durante o processo de adaptação, CountVectorizer selecionará as principais palavras do vocabSize ordenadas por frequência de termo no corpus. Um parâmetro opcional minDF também afeta o processo de adaptação, especificando o número mínimo (ou fração se <1,0) de documentos em que um termo deve aparecer para ser incluído no vocabulário. Outro parâmetro de alternância binário opcional controla o vetor de saída. Se definido como verdadeiro, todas as contagens diferentes de zero são definidas como 1. Isso é especialmente útil para modelos probabilísticos discretos que modelam contagens binárias, em vez de inteiras.

In [15]:
from pyspark.ml.feature import CountVectorizer

# Dados de entrada: cada linha é um pacote de palavras com um ID.
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

# ajustar um CountVectorizerModel do corpus.
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

model = cv.fit(df)

result = model.transform(df)
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

